In [ ]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from IPython.display import Audio, display,Markdown, display, update_display
import gradio as gr

In [ ]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"






In [ ]:
load_dotenv(override=True)
openai_key = os.getenv("OPENAI_API_KEY")


if openai_key:
    print(f"OpenAI API Key exists and begins {openai_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
openai = OpenAI()

In [ ]:


system_message="You are a expert in technical question please responds with an explanation"
system_message += "Give short, courteous answers, no more than 5 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

def message_gpt(prompt):
    messages= [{
        "role":"system","content" : system_message
    },{
        "role":"user","content": prompt
    }]

    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def message_ollama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in response:
        # kolla att 'delta' finns
        delta = getattr(chunk.choices[0], "delta", None)
        if delta and hasattr(delta, "content"):
            result += delta.content
            yield result

In [ ]:
def change_model(prompt,model):
    if model =="OpenAi":
        result = message_gpt(prompt)
    elif model == "Ollama":
        result = message_ollama(prompt)
    else:
        raise ValueError("Uknow model")
    yield from result

In [ ]:
gr.Interface(fn=change_model,
             inputs=[gr.Textbox(label="Your massage"), gr.Dropdown(["OpenAi", "Ollama"], label="Select model" , value="OpenAi")],
             outputs=[gr.Markdown(label="Response: ")] ,
             flagging_mode="never").launch()


In [ ]:
MODELGPT = "gpt-4o-mini"

programing_level= {"python":"easy","javascript":"easy","C":"hard","C#":"intermediate"}

def get_level(programming):
     print(f"Tool get_level called for {programming}")
     if not programming:  # Kollar om booking är None eller tom sträng
        return "No booking info provided"
     level = programming.lower()
     return programing_level.get(level,"Unknown")



In [ ]:
level_function = {
    "name": "get_level",
    "description": "Get the level of a programming language for technical question . Call this whenever you need to know the level programming, for example when a customer asks 'How hard is a programming Python'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The programing_level that the customer wants to know ",
            },
        },
        "required": ["programming"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": level_function}]

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    level = arguments.get('programming')

    if not level:   # Hantera None eller tom sträng
        level = ""
    programming = get_level(level)
    response = {
        "role": "tool",
        "content": json.dumps({"programming": level,"programming": programming}),
        "tool_call_id": tool_call.id
    }
    return response

In [ ]:
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="sage",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODELGPT, messages=messages, tools=tools)
    
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODELGPT, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

   
    talker(reply)
    
    return history

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
    with gr.Row():
        entry = gr.Textbox(label="Chat with your Ai Assistans:")
    with gr.Row():
        clear = gr.Button("Clear")
    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=chatbot
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch()